In [1]:
import os
import pandas as pd
import ftplib

In [2]:
def getFile(ftp, base_path, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(os.path.join(base_path,filename), 'wb').write)
    except:
        print("Error")

In [3]:
#
# import speciment inventory from BCDC
#
# https://github.com/AllenInstitute/data_wrangling/tree/dev/curation_sample_inventory/fMOST_brains
#
inventory_directory = 'C:/Users/lydia/OneDrive/Work/data_wrangling/curation_sample_inventory/fMOST_brains'
sample_inventory_file = os.path.join( inventory_directory, '20210907_Zeng_fMOST_Specimen_Inventory.csv')
sdf = pd.read_csv( sample_inventory_file )

In [4]:
#
# temporary fix the "Sample ID" to substitute a "dash" for the second underscore
#
slist = sdf['Sample ID'].values
slist = [s.split('_') for s in slist]
sdf['Sample ID'] = [s[0]+'_'+s[1]+'-'+s[2] for s in slist]
sdf['AIBS LABTrack ID'] = [s[1] for s in slist]
sdf['HUST Sample ID'] = [s[2] for s in slist]

In [5]:
# make 'Sample ID' the data frame index
sdf.set_index('Sample ID',inplace=True)

In [6]:
#
# setup output directory
# 
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'

In [7]:
sdf.head(10)

,Sample Type (CV),Species (CV),Species NCBI Taxonomy ID,Parent Specimen ID,Parent Specimen Type (CV),Subject ID,Age,Sex (CV),Genotype,Modality (CV),...,Organization (CV),Investigator,Grant Number (CV),Data Collection (CV),R24 Name (CV),R24 link,Comments,Metadata Submission,AIBS LABTrack ID,HUST Sample ID
Sample ID,,,,,,,,,,,,,,,,,,,,,
mouseID_210254-15257,whole brain,mouse,NCBI:txid10090,NaN,NaN,210254,22w,female,Pvalb-T2A-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)/wt,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q4,210254,15257
mouseID_236174-16124,whole brain,mouse,NCBI:txid10090,NaN,NaN,236174,21w,male,Gnb4-IRES2-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,236174,16124
mouseID_297974-17109,whole brain,mouse,NCBI:txid10090,NaN,NaN,297974,14w,male,Gnb4-IRES2-CreERT2/wt;Ai140(TIT2L-GFP-ICL-tTA2...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,297974,17109
mouseID_314107-17300,whole brain,mouse,NCBI:txid10090,NaN,NaN,314107,48w,male,Plxnd1-CreER/wt;Ai82(TITL-GFP)/Ai140(TIT2L-GFP...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,314107,17300
mouseID_321237-17302,whole brain,mouse,NCBI:txid10090,NaN,NaN,321237,29w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,321237,17302
mouseID_321244-17545,whole brain,mouse,NCBI:txid10090,NaN,NaN,321244,16w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,321244,17545
mouseID_325875-17543,whole brain,mouse,NCBI:txid10090,NaN,NaN,325875,48w,male,Tlx3-Cre_PL56/wt;Ai82(TITL-GFP)/Ai161(TIT2L-GF...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q4,325875,17543
mouseID_326952-17304,whole brain,mouse,NCBI:txid10090,NaN,NaN,326952,18w,male,Tlx3-Cre_PL56/wt;Ai82(TITL-GFP)/Ai161(TIT2L-GF...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,326952,17304
mouseID_339951-17781,whole brain,mouse,NCBI:txid10090,NaN,NaN,339951,19w,female,Gnb4-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TIT...,cell morphology,...,Allen Institute for Brain Science,Hongkui Zeng,1U19MH114830-01,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,339951,17781


In [8]:
ftp = ftplib.FTP(host="download.brainimagelibrary.org")
ftp.login()

home_dir = ftp.pwd()
remote_base = "biccn/zeng/luo/fMOST"

sdf['directories'] = ''
sdf['txt_files'] = ''
sdf['json_files'] = ''

for specimen, row in sdf.iterrows():
    
    print(specimen)
    
    # create subdirectory if it doesn't already exists
    spath = os.path.join( output_directory, specimen )
    if not os.path.exists( spath ) :
        os.makedirs( spath )
    
    # get listing of remote directory
    rpath = remote_base + "/" + specimen
    ftp.cwd( rpath )
    
    data = []
    ftp.dir(data.append)
    
    data = [x.split() for x in data if x != '']
    data = pd.DataFrame(data)

    
    #print(data)
    
    # find the directories
    tdf = data[data[0].str.contains('d')]
    sdf.at[specimen,'directories'] = ';'.join(tdf[8].values)

    # find the .txt files
    tdf = data[data[8].str.contains('.txt')]
    sdf.at[specimen,'txt_files'] = ';'.join(tdf[8].values)    
    
    # find the .json files
    tdf = data[data[8].str.contains('.json')]
    sdf.at[specimen,'json_files'] = ';'.join(tdf[8].values) 
    
    # download the .txt/.json files
    for f in sdf.at[specimen,'txt_files'].split(';') :
        if f != '' :
            getFile(ftp, spath, f)
        
    for f in sdf.at[specimen,'json_files'].split(';') :
        if f != '' :
            getFile(ftp, spath, f)    
    
    
    # change back to home directory
    ftp.cwd( home_dir )

    
ftp.quit()

mouseID_210254-15257
mouseID_236174-16124
mouseID_297974-17109
mouseID_314107-17300
mouseID_321237-17302
mouseID_321244-17545
mouseID_325875-17543
mouseID_326952-17304
mouseID_339951-17781
mouseID_339952-17782
mouseID_351327-17787
mouseID_351331-17788
mouseID_355458-18053
mouseID_355459-18054
mouseID_358361-18047
mouseID_360835-18049
mouseID_362188-191815
mouseID_362191-191816
mouseID_367667-18052
mouseID_373187-191817
mouseID_373367-18454
mouseID_373368-18455
mouseID_373641-18462
mouseID_374706-18461
mouseID_374707-18452
mouseID_375394-18468
mouseID_377387-18466
mouseID_378667-18469
mouseID_378668-18470
mouseID_380470-191812
mouseID_380471-191813
mouseID_381484-18457
mouseID_381487-18458
mouseID_381488-18464
mouseID_383128-18465
mouseID_383680-18463
mouseID_394528-18867
mouseID_404421-182720
mouseID_405426-182724
mouseID_405429-182725
mouseID_411732-182718
mouseID_417570-182721
mouseID_417571-182722
mouseID_418245-182727
mouseID_420489-191801
mouseID_423019-191803
mouseID_426124-19180

'221 Goodbye.'

In [9]:
sdf.head(10)

,Sample Type (CV),Species (CV),Species NCBI Taxonomy ID,Parent Specimen ID,Parent Specimen Type (CV),Subject ID,Age,Sex (CV),Genotype,Modality (CV),...,Data Collection (CV),R24 Name (CV),R24 link,Comments,Metadata Submission,AIBS LABTrack ID,HUST Sample ID,directories,txt_files,json_files
Sample ID,,,,,,,,,,,,,,,,,,,,,
mouseID_210254-15257,whole brain,mouse,NCBI:txid10090,NaN,NaN,210254,22w,female,Pvalb-T2A-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)/wt,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q4,210254,15257,.;..;green;red,,
mouseID_236174-16124,whole brain,mouse,NCBI:txid10090,NaN,NaN,236174,21w,male,Gnb4-IRES2-CreERT2/wt;Ai139(TIT2L-GFP-ICL-TPT)...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,236174,16124,.;..;green;red,,
mouseID_297974-17109,whole brain,mouse,NCBI:txid10090,NaN,NaN,297974,14w,male,Gnb4-IRES2-CreERT2/wt;Ai140(TIT2L-GFP-ICL-tTA2...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,297974,17109,.;..;green;red,,
mouseID_314107-17300,whole brain,mouse,NCBI:txid10090,NaN,NaN,314107,48w,male,Plxnd1-CreER/wt;Ai82(TITL-GFP)/Ai140(TIT2L-GFP...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,314107,17300,.;..;green;red,,
mouseID_321237-17302,whole brain,mouse,NCBI:txid10090,NaN,NaN,321237,29w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,321237,17302,.;..;green;red,,
mouseID_321244-17545,whole brain,mouse,NCBI:txid10090,NaN,NaN,321244,16w,male,Tnnt1-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TI...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,321244,17545,.;..;green;red,,
mouseID_325875-17543,whole brain,mouse,NCBI:txid10090,NaN,NaN,325875,48w,male,Tlx3-Cre_PL56/wt;Ai82(TITL-GFP)/Ai161(TIT2L-GF...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q4,325875,17543,.;..;green;red,17543-metainfo.txt;metainfo.txt,17543.json
mouseID_326952-17304,whole brain,mouse,NCBI:txid10090,NaN,NaN,326952,18w,male,Tlx3-Cre_PL56/wt;Ai82(TITL-GFP)/Ai161(TIT2L-GF...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,326952,17304,.;..;green;red,,
mouseID_339951-17781,whole brain,mouse,NCBI:txid10090,NaN,NaN,339951,19w,female,Gnb4-IRES2-CreERT2/wt;Ai82(TITL-GFP)/Ai140(TIT...,cell morphology,...,zeng_fmost,BIL,https://download.brainimagelibrary.org/biccn/z...,NaN,2019Q1,339951,17781,.;..;green;red,,


In [10]:
#
# write out data frame
#
output_path = os.path.join(output_directory,'sample_inventory_with_directory_info.csv')
sdf.to_csv( output_path )